# Data Cleaning
### Loading the files and removing blank data

First we import the required libraries and adjust out display settings to more easily view the datasets.

In [19]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 30)

Read each dataset as a Pandas dataframe.

In [20]:
df_bom = pd.read_csv('data/bom.movie_gross.csv')

df_iname = pd.read_csv('data/imdb.name.basics.csv')
df_iakas = pd.read_csv('data/imdb.title.akas.csv')
df_ititlebas = pd.read_csv('data/imdb.title.basics.csv')
df_ititlecrew = pd.read_csv('data/imdb.title.crew.csv')
df_ititlepri = pd.read_csv('data/imdb.title.principals.csv')
df_ititlerate = pd.read_csv('data/imdb.title.ratings.csv')

df_imovtxt = pd.read_csv('data/rt.movie_info.tsv', delimiter='\t') # utf-8 is default
df_ireviewtxt = pd.read_csv('data/rt.reviews.tsv', sep='\t',  encoding='ISO-8859-1', low_memory=False)

df_rtmov = pd.read_csv('data/tmdb.movies.csv') # utf-8 is default
df_rt_budget = pd.read_csv('data/tn.movie_budgets.csv') # utf-8 is default


# df_copy = df.copy() # make a copy of the data in the event we clobber



In [ ]:
df_bom.info()
df_iname.info()
df_iakas.info()
df_ititlebas.info()
df_ititlecrew.info()
df_ititlepri.info()
df_ititlerate.info()

df_imovtxt.info()
df_ireviewtxt.info()

df_rtmov.info()
df_rt_budget.info()


In [22]:
df_studio_gross = df_bom.copy(deep="True")

In [23]:
df_studio_gross.drop(['title', 'year'], axis=1, inplace=True)


In [59]:
df_studio_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 3 columns):
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
dtypes: float64(1), object(2)
memory usage: 68.7+ KB


### drop nulls and convert domestic_gross to integer

In [41]:
df_studio_gross['foreign_gross'].isnull().sum()


1350

In [42]:
df_studio_gross['foreign_gross'].dropna(inplace=True)

In [43]:
df_studio_gross['foreign_gross'].isnull().sum()

0

In [58]:
df_studio_gross.foreign_gross.astype('int32')

ValueError: invalid literal for int() with base 10: '1,131.6'

In [48]:
df_studio_gross['domestic_gross'].isnull().sum()

28

In [49]:
df_studio_gross['domestic_gross'].dropna(inplace=True)

In [50]:
df_studio_gross['domestic_gross'].isnull().sum()

0

In [55]:
df_studio_gross.domestic_gross.astype('int32')

0       415000000
1       334200000
2       296000000
3       292600000
4       238700000
          ...    
3382         6200
3383         4800
3384         2500
3385         2400
3386         1700
Name: domestic_gross, Length: 3359, dtype: int32

In [32]:
df_studio_gross['total_gross'] = df_studio_gross['domestic_gross'] + df_studio_gross['foreign_gross']


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [28]:
type(df_studio_gross['foreign_gross'])

pandas.core.series.Series

In [6]:
!pwd

/Users/kennedy/Documents/GitHub/fiprojects/mod_1_movie


In [35]:
df_rt_budget.columns

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [36]:
df_rt_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [37]:
df_rt_budget.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [38]:
df_rt_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [50]:
df_working = df_rt_budget.copy(deep="True") # make a hard copy of the dataset to work with

In [82]:
df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,PnL
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,999721747


#### Clean the characters from the currency columns and convert to integer for calculations and graphing

In [52]:

df_working[df_working.columns[3:]] = df_working[df_working.columns[3:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)

In [53]:
df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [57]:
df_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null int64
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


In [59]:
df_working['foreign_gross'] = df_working['worldwide_gross'] - df_working['domestic_gross']

In [60]:
df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365


In [69]:
df_working['PnL'] = df_working['worldwide_gross'] - df_working['production_budget']

In [72]:
df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,PnL
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,999721747


#### Save the dataset to a file that we will access in separate notebook

In [73]:
df_working.to_csv('data/tn.movie_budgets_working.csv')

In [ ]:
df_iname = pd.read_csv('data/imdb.name.basics.csv')
df_iakas = pd.read_csv('data/imdb.title.akas.csv')
df_ititlebas = pd.read_csv('data/imdb.title.basics.csv')
df_ititlecrew = pd.read_csv('data/imdb.title.crew.csv')
df_ititlepri = pd.read_csv('data/imdb.title.principals.csv')
df_ititlerate = pd.read_csv('data/imdb.title.ratings.csv')

In [68]:
df_ititlerate.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [83]:
df_ititlebas.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [74]:
df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,PnL
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,999721747


#### Merge basics with ratings on column name tconst

In [84]:
df_merged = pd.merge(df_ititlerate, df_ititlebas, on='tconst')

In [97]:
df_iname.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [96]:
df_ititlepri.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [ ]:
df_imovtxt
df_ireviewtxt

In [111]:
df_imovtxt.columns

Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

In [112]:
df_ireviewtxt.columns


Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')

In [113]:
df_rtmov.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [114]:
df_rtmov.describe()

,Unnamed: 0,id,popularity,vote_average,vote_count
count,26517.00000,26517.000000,26517.000000,26517.000000,26517.000000
mean,13258.00000,295050.153260,3.130912,5.991281,194.224837
std,7654.94288,153661.615648,4.355229,1.852946,960.961095
min,0.00000,27.000000,0.600000,0.000000,1.000000
25%,6629.00000,157851.000000,0.600000,5.000000,2.000000
50%,13258.00000,309581.000000,1.374000,6.000000,5.000000
75%,19887.00000,419542.000000,3.694000,7.000000,28.000000
max,26516.00000,608444.000000,80.773000,10.000000,22186.000000
